# Compare MendelImpute against Minimac4 and Beagle5 on simulated data

In [1]:
using Revise
using VCFTools
using MendelImpute
using GeneticVariation
using Random
using SparseArrays
using JLD2, FileIO, JLSO
using ProgressMeter
using GroupSlices
using ThreadPools
# using Plots
# using ProfileView

┌ Info: Precompiling MendelImpute [e47305d1-6a61-5370-bc5d-77554d143183]
└ @ Base loading.jl:1273


# Simulate data

### Step 0. Install `msprime`

[msprime download Link](https://msprime.readthedocs.io/en/stable/installation.html).

Some people might need to activate conda environment via `conda config --set auto_activate_base True`. You can turn it off once simulation is done by executing `conda config --set auto_activate_base False`.


### Step 1. Simulate data in terminal

```
python3 msprime_script.py 4000 10000 5000000 2e-8 2e-8 2019 > full.vcf
```

Arguments: 
+ Number of haplotypes = 40000
+ Effective population size = 10000 ([source](https://www.the-scientist.com/the-nutshell/ancient-humans-more-diverse-43556))
+ Sequence length = 10 million (same as Beagle 5's choice)
+ Rrecombination rate = 2e-8 (default)
+ mutation rate = 2e-8 (default)
+ seed = 2019

### Step 2: Convert simulated haplotypes to reference haplotypes and target genotype files

+ `haplo_ref.vcf.gz`: haplotype reference files
+ `target.vcf.gz`: complete genotype information
+ `target_masked.vcf.gz`: the same as `target.vcf.gz` except some entries are masked

In [4]:
records, samples = nrecords("full.vcf"), nsamples("full.vcf")
@show records
@show samples;

# compute target and reference index
tgt_index = falses(samples)
tgt_index[samples-999:end] .= true
ref_index = .!tgt_index
record_index = trues(records) # save all records (SNPs) 

# create target.vcf.gz and haplo_ref.vcf.gz
@time VCFTools.filter("full.vcf", record_index, tgt_index, des = "target.vcf.gz")
@time VCFTools.filter("full.vcf", record_index, ref_index, des = "haplo_ref.vcf.gz")

# import full target matrix. Also transpose so that columns are samples. 
@time X = convert_gt(Float32, "target.vcf.gz"; as_minorallele=false)
X = copy(X')

# mask 10% entries
p, n = size(X)
Random.seed!(123)
missingprop = 0.1
X .= ifelse.(rand(Float32, p, n) .< missingprop, missing, X)
masks = ismissing.(X)

# save X to new VCF file
mask_gt("target.vcf.gz", masks, des="target_masked.vcf.gz")

records = 35897
samples = 2000
 70.297829 seconds (397.28 M allocations: 33.310 GiB, 11.51% gc time)
 67.404677 seconds (395.78 M allocations: 33.237 GiB, 11.69% gc time)
 18.210343 seconds (144.39 M allocations: 12.666 GiB, 17.95% gc time)


# Try compressing haplotype ref panels

In [34]:
# compress as jld2
reffile = "haplo_ref.vcf.gz"
tgtfile = "target_masked.vcf.gz"
outfile = "haplo_ref.jld2"
width = 512
@time compress_haplotypes(reffile, tgtfile, outfile, width);

importing reference data...100%|████████████████████████| Time: 0:00:07
Importing genotype file...100%|█████████████████████████| Time: 0:00:06


 17.346864 seconds (146.87 M allocations: 11.060 GiB, 10.62% gc time)


In [35]:
# compress as jlso
reffile = "haplo_ref.vcf.gz"
tgtfile = "target_masked.vcf.gz"
outfile = "haplo_ref.jlso"
width = 512
@time compress_haplotypes(reffile, tgtfile, outfile, width);

importing reference data...100%|████████████████████████| Time: 0:00:06
Importing genotype file...100%|█████████████████████████| Time: 0:00:06


 15.859604 seconds (145.08 M allocations: 11.037 GiB, 9.12% gc time)


In [5]:
function compress()
    widths  = [32, 64, 128, 256, 512]
    reffile = "haplo_ref.vcf.gz"
    tgtfile = "target_masked.vcf.gz"
    H, H_sampleID, H_chr, H_pos, H_ids, H_ref, H_alt = convert_ht(Bool, reffile, trans=true, save_snp_info=true, msg="importing reference data...")
    X, X_sampleID, X_chr, X_pos, X_ids, X_ref, X_alt = VCFTools.convert_gt(UInt8, tgtfile, trans=true, save_snp_info=true, msg = "Importing genotype file...")
    for width in widths
        outfile = "haplo_ref.w$width.jlso"
        @time compress_haplotypes(H, X, outfile, X_pos, H_sampleID, H_chr, H_pos, H_ids, H_ref, H_alt, width)
    end
end
compress()

importing reference data...100%|████████████████████████| Time: 0:00:07
Importing genotype file...100%|█████████████████████████| Time: 0:00:06


  8.725367 seconds (8.18 M allocations: 2.480 GiB, 5.05% gc time)
  4.355800 seconds (834.58 k allocations: 1.124 GiB, 2.44% gc time)
  2.984843 seconds (698.91 k allocations: 617.586 MiB, 2.20% gc time)
  2.543171 seconds (582.07 k allocations: 358.278 MiB, 1.72% gc time)


In [36]:
# load jld2
@time @load "haplo_ref.jld2" compressed_Hunique;

  0.100215 seconds (774.49 k allocations: 58.377 MiB)


In [37]:
# load jlso
@time loaded = JLSO.load("haplo_ref.jlso")
compressed_Hunique = loaded[:compressed_Hunique];

  0.180459 seconds (611.68 k allocations: 38.414 MiB)


In [28]:
;ls -al haplo_ref.jld2

-rw-r--r--  1 biona001  staff  24958314 Jun 28 22:49 haplo_ref.jld2


In [29]:
;ls -al haplo_ref.jlso

-rw-r--r--  1 biona001  staff  1994565 Jun 28 22:48 haplo_ref.jlso


In [30]:
;ls -al haplo_ref.vcf.gz

-rw-r--r--@ 1 biona001  staff  5449864 Apr  5 19:59 haplo_ref.vcf.gz


# Haplotype thinning (experiment)

In [11]:
# haplopair_thin (repeated BLAS 2), keep = 100 (1 thread)
Random.seed!(2020)
tgtfile = "target_masked.vcf.gz"
reffile = "haplo_ref.jlso"
outfile = "imputed_target.vcf.gz"
width   = 512
@time hs, ph = phase(tgtfile, reffile, outfile = outfile, width = width, thinning_factor=100);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "target.vcf.gz")
n, p = size(X_mendel)
error_rate = sum(X_mendel .!= X_complete) / n / p

Importing genotype file...100%|█████████████████████████| Time: 0:00:07
Computing optimal haplotype pairs...100%|███████████████| Time: 0:00:18
Merging breakpoints...100%|█████████████████████████████| Time: 0:00:09


Total windows = 70, averaging ~ 324 unique haplotypes per window.

Timings: 
    Data import                     = 7.83421 seconds
    Computing haplotype pair        = 18.4104 seconds
        BLAS3 mul! to get M and N      = 15.7233 seconds per thread
        haplopair search               = 0.487475 seconds per thread
        supplying constant terms       = 0.0141915 seconds per thread
        finding redundant happairs     = 0.298754 seconds per thread
    Phasing by dynamic programming  = 9.17926 seconds
    Imputation                      = 3.22604 seconds

 38.650041 seconds (73.96 M allocations: 6.631 GiB, 2.29% gc time)


0.00012427222330556873

In [9]:
# haplopair_thin (repeated BLAS 2), keep = 100 (1 thread)
Random.seed!(2020)
tgtfile = "target_masked.vcf.gz"
reffile = "haplo_ref.jlso"
outfile = "imputed_target.vcf.gz"
width   = 512
@time hs, ph = phase(tgtfile, reffile, outfile = outfile, width = width, thinning_factor=100);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
# X_complete = convert_gt(Float32, "target.vcf.gz")
n, p = size(X_mendel)
error_rate = sum(X_mendel .!= X_complete) / n / p

Importing genotype file...100%|█████████████████████████| Time: 0:00:06
Merging breakpoints...100%|█████████████████████████████| Time: 0:00:08


Total windows = 70, averaging ~ 324 unique haplotypes per window.

Timings: 
    Data import                     = 7.39464 seconds
    Computing haplotype pair        = 4.43691 seconds
        BLAS3 mul! to get M and N      = 0.304902 seconds per thread
        haplopair search               = 2.99792 seconds per thread
        supplying constant terms       = 0.0361403 seconds per thread
        finding redundant happairs     = 0.252659 seconds per thread
    Phasing by dynamic programming  = 8.0823 seconds
    Imputation                      = 3.00531 seconds

 22.919910 seconds (73.96 M allocations: 6.726 GiB, 3.88% gc time)


0.0001239657910131766

# MendelImpute error

In [7]:
# keep best pair only (1 thread)
Random.seed!(2020)
width   = 64
tgtfile = "target_masked.vcf.gz"
reffile = "haplo_ref.w$width.jlso"
outfile = "imputed_target.vcf.gz"
@time hs, ph = phase(tgtfile, reffile, outfile = outfile, width = width);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "target.vcf.gz")
n, p = size(X_mendel)
error_rate = sum(X_mendel .!= X_complete) / n / p

Importing genotype file...100%|█████████████████████████| Time: 0:00:07
Computing optimal haplotype pairs...100%|███████████████| Time: 0:00:12
Merging breakpoints...100%|█████████████████████████████| Time: 0:08:28


Total windows = 560, averaging ~ 57 unique haplotypes per window.

Timings: 
    Data import                     = 8.23081 seconds
    Computing haplotype pair        = 12.2929 seconds
        BLAS3 mul! to get M and N      = 0.0804824 seconds per thread
        haplopair search               = 0.718569 seconds per thread
        supplying constant terms       = 0.0266793 seconds per thread
        finding redundant happairs     = 6.70131 seconds per thread
    Phasing by dynamic programming  = 508.801 seconds
    Imputation                      = 3.5402 seconds

532.932116 seconds (87.42 M allocations: 11.146 GiB, 0.63% gc time)


0.00130353511435496

In [44]:
# keep best pair only (1 thread)
Random.seed!(2020)
tgtfile = "target_masked.vcf.gz"
reffile = "haplo_ref.jlso"
outfile = "imputed_target.vcf.gz"
width   = 512
@time hs, ph = phase(tgtfile, reffile, outfile = outfile, width = width);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "target.vcf.gz")
n, p = size(X_mendel)
error_rate = sum(X_mendel .!= X_complete) / n / p

Importing genotype file...100%|█████████████████████████| Time: 0:00:07
Merging breakpoints...100%|█████████████████████████████| Time: 0:00:08


Total windows = 70, averaging ~ 324 unique haplotypes per window.

Timings: 
    Data import                     = 7.53159 seconds
    Computing haplotype pair        = 3.74268 seconds
        BLAS3 mul! to get M and N      = 0.102913 seconds per thread
        haplopair search               = 2.36716 seconds per thread
        supplying constant terms       = 0.0350019 seconds per thread
        finding redundant happairs     = 0.25264 seconds per thread
    Phasing by dynamic programming  = 8.01037 seconds
    Imputation                      = 3.143 seconds

 22.427998 seconds (73.66 M allocations: 6.632 GiB, 5.34% gc time)


0.00012268434688135499

In [60]:
# keep top matching happairs (1 thread)
Random.seed!(2020)
tgtfile = "target_masked.vcf.gz"
reffile = "haplo_ref.jlso"
outfile = "imputed_target.vcf.gz"
width   = 512
@time hs, ph = phase(tgtfile, reffile, outfile = outfile, width = width, rescreen=true);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "target.vcf.gz")
n, p = size(X_mendel)
error_rate = sum(X_mendel .!= X_complete) / n / p

Importing genotype file...100%|█████████████████████████| Time: 0:00:08
Computing optimal haplotype pairs...100%|███████████████| Time: 0:00:07
Merging breakpoints...100%|█████████████████████████████| Time: 0:00:09


Total windows = 70, averaging ~ 324 unique haplotypes per window.

Timings: 
    Data import                     = 8.60848 seconds
    Computing haplotype pair        = 7.25466 seconds
        BLAS3 mul! to get M and N      = 0.0993171 seconds (on thread 1)
        haplopair search               = 4.07569 seconds (on thread 1)
        min least sq on observed data  = 1.16035 seconds (on thread 1)
        finding redundant happairs     = 0.525039 seconds (on thread 1)
    Phasing by dynamic programming  = 9.25942 seconds
    Imputation                      = 3.39352 seconds

 28.516151 seconds (73.61 M allocations: 7.466 GiB, 7.49% gc time)


0.0001033791124606513

In [3]:
# keep best pair only (8 thread)
Random.seed!(2020)
tgtfile = "target_masked.vcf.gz"
reffile = "haplo_ref.jlso"
outfile = "imputed_target.vcf.gz"
width   = 512
@time hs, ph = phase(tgtfile, reffile, outfile = outfile, width = width);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "target.vcf.gz")
n, p = size(X_mendel)
error_rate = sum(X_mendel .!= X_complete) / n / p

Importing genotype file...100%|█████████████████████████| Time: 0:00:06


Total window = 70, each with ~ 324 unique haplotypes on avg

Timings: 
    Data import                     = 6.64381 seconds
    Computing haplotype pair        = 1.17348 seconds
        BLAS3 mul! to get M and N      = 0.0281553 seconds (on thread 1)
        haplopair search               = 0.652449 seconds (on thread 1)
        supplying constant terms       = 0.00478852 seconds (on thread 1)
        finding redundant happairs     = 0.093422 seconds (on thread 1)
    Phasing by dynamic programming  = 1.45871 seconds
    Imputation                      = 2.8129 seconds
 12.089255 seconds (73.53 M allocations: 7.401 GiB, 7.49% gc time)


0.00012271220436248155

In [3]:
# Keep a list of top haplotype pairs (1 thread)
Random.seed!(2020)
tgtfile = "target_masked.vcf.gz"
reffile = "haplo_ref.jlso"
outfile = "imputed_target.vcf.gz"
width   = 500
@time hs, ph = phase(tgtfile, reffile, outfile = outfile, width = width);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "target.vcf.gz")
n, p = size(X_mendel)
error_rate = sum(X_mendel .!= X_complete) / n / p

Importing genotype file...100%|█████████████████████████| Time: 0:00:06
Computing optimal haplotype pairs...100%|███████████████| Time: 0:00:10
Merging breakpoints...100%|█████████████████████████████| Time: 0:00:25


Data import time                    = 6.6587 seconds
Computing haplotype pair time       = 10.2962 seconds
Phasing by dynamic programming time = 25.9828 seconds
Imputing time                       = 3.6637 seconds
 46.601150 seconds (73.68 M allocations: 7.879 GiB, 2.17% gc time)


0.00010181909351756413

In [5]:
# Keep a list of top haplotype pairs (8 thread)
Random.seed!(2020)
tgtfile = "target_masked.vcf.gz"
reffile = "haplo_ref.jlso"
outfile = "imputed_target.vcf.gz"
width   = 500
@time hs, ph = phase(tgtfile, reffile, outfile = outfile, width = width);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "target.vcf.gz")
n, p = size(X_mendel)
error_rate = sum(X_mendel .!= X_complete) / n / p

Importing genotype file...100%|█████████████████████████| Time: 0:00:06


Data import time                    = 7.00521 seconds
Computing haplotype pair time       = 2.14302 seconds
Phasing by dynamic programming time = 4.11389 seconds
Imputing time                       = 3.09487 seconds
 16.357013 seconds (73.69 M allocations: 7.939 GiB, 5.88% gc time)


0.00010181909351756413

# MendelImpute with intersecting haplotype sets

In [9]:
# keep best pair only (1 thread)
Random.seed!(2020)
width   = 64
tgtfile = "target_masked.vcf.gz"
reffile = "haplo_ref.w$width.jlso"
outfile = "imputed_target.vcf.gz"
@time hs, ph = phase(tgtfile, reffile, outfile = outfile, width = width, 
    dynamic_programming = false);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "target.vcf.gz")
n, p = size(X_mendel)
error_rate = sum(X_mendel .!= X_complete) / n / p

Importing genotype file...100%|█████████████████████████| Time: 0:00:07


Total windows = 560, averaging ~ 57 unique haplotypes per window.

Timings: 
    Data import                     = 7.87053 seconds
    Computing haplotype pair        = 2.28208 seconds
        BLAS3 mul! to get M and N      = 0.0803106 seconds per thread
        haplopair search               = 0.698855 seconds per thread
        supplying constant terms       = 0.0258972 seconds per thread
        finding redundant happairs     = 0.671479 seconds per thread
    Phasing by dynamic programming  = 0.858593 seconds
    Imputation                      = 3.18644 seconds

 14.198042 seconds (78.40 M allocations: 6.726 GiB, 10.04% gc time)


0.0013129230854946096

In [47]:
# keep best pair only (1 thread)
Random.seed!(2020)
tgtfile = "target_masked.vcf.gz"
reffile = "haplo_ref.jlso"
outfile = "imputed_target.vcf.gz"
width   = 512
@time hs, ph = phase(tgtfile, reffile, outfile = outfile, width = width,
    dynamic_programming = false);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "target.vcf.gz")
n, p = size(X_mendel)
error_rate = sum(X_mendel .!= X_complete) / n / p

Importing genotype file...100%|█████████████████████████| Time: 0:00:07


Total windows = 70, averaging ~ 324 unique haplotypes per window.

Timings: 
    Data import                     = 7.65154 seconds
    Computing haplotype pair        = 3.42246 seconds
        BLAS3 mul! to get M and N      = 0.0976265 seconds per thread
        haplopair search               = 2.44649 seconds per thread
        supplying constant terms       = 0.0362332 seconds per thread
        finding redundant happairs     = 0.0447599 seconds per thread
    Phasing by dynamic programming  = 0.127417 seconds
    Imputation                      = 3.12078 seconds

 14.322548 seconds (73.73 M allocations: 6.138 GiB, 6.11% gc time)


0.00014143243167952753

# Beagle 5 Error

In [15]:
# run beagle 5 and import imputed data 
run(`java -jar beagle.28Sep18.793.jar gt=target_masked.vcf.gz ref=haplo_ref.vcf.gz out=beagle.result`)

# beagle 5 error rate
X_beagle = convert_gt(Float32, "beagle.result.vcf.gz", as_minorallele=false)
error_rate = sum(X_beagle .!= X_complete) / n / p

beagle.28Sep18.793.jar (version 5.0)
Copyright (C) 2014-2018 Brian L. Browning
Enter "java -jar beagle.28Sep18.793.jar" to list command line argument
Start time: 11:01 AM PST on 23 Jan 2020

Command line: java -Xmx3641m -jar beagle.28Sep18.793.jar
  gt=target_masked.vcf.gz
  ref=haplo_ref.vcf.gz
  out=beagle.result

No genetic map is specified: using 1 cM = 1 Mb

Reference samples:       1,000
Study samples:           1,000

Window 1 (1:36-4999683)
Reference markers:      35,897
Study markers:          35,897

Burnin  iteration 1:           50 seconds
Burnin  iteration 2:           32 seconds
Burnin  iteration 3:           13 seconds
Burnin  iteration 4:           18 seconds
Burnin  iteration 5:           23 seconds
Burnin  iteration 6:           25 seconds

Phasing iteration 1:           13 seconds
Phasing iteration 2:           13 seconds
Phasing iteration 3:           13 seconds
Phasing iteration 4:           12 seconds
Phasing iteration 5:           12 seconds
Phasing iteration 6: 

2.231384238237179e-5

# Minimac4 error

Need to first convert reference vcf file to m3vcf using minimac3 (on Hoffman)

```Julia
minimac3 = "/u/home/b/biona001/haplotype_comparisons/minimac3/Minimac3/bin/Minimac3"
@time run(`$minimac3 --refHaps haplo_ref.vcf.gz --processReference --prefix haplo_ref`)
```

In [17]:
# run minimac 4
minimac4 = "/Users/biona001/Benjamin_Folder/UCLA/research/softwares/Minimac4/build/minimac4"
run(`$minimac4 --refHaps haplo_ref.m3vcf.gz --haps target_masked.vcf.gz --prefix minimac4.result`)
    
X_minimac = convert_gt(Float32, "minimac4.result.dose.vcf.gz", as_minorallele=false)
error_rate = sum(X_minimac .!= X_complete) / n / p



 -------------------------------------------------------------------------------- 
          Minimac4 - Fast Imputation Based on State Space Reduction HMM
 --------------------------------------------------------------------------------
           (c) 2014 - Sayantan Das, Christian Fuchsberger, David Hinds
                             Mary Kate Wing, Goncalo Abecasis 

 Version: 1.0.2;
 Built: Mon Sep 30 11:52:22 PDT 2019 by biona001

 Command Line Options: 
       Reference Haplotypes : --refHaps [haplo_ref.m3vcf.gz], --passOnly,
                              --rsid, --referenceEstimates [ON],
                              --mapFile [docs/geneticMapFile.b38.map.txt.gz]
          Target Haplotypes : --haps [target_masked.vcf.gz]
          Output Parameters : --prefix [minimac4.result], --estimate,
                              --nobgzip, --vcfBuffer [200], --format [GT,DS],
                              --allTypedSites, --meta, --memUsage
        Chunking Parameters : --ChunkLengthMb

0.00018399866284090594